In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


# DATA PREPROCESSING TRAIN

### LOAD DATA

In [59]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"harismohomed","key":"defb1a6bd72ffb88a8c9e259faae5a31"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [60]:
!pip install kaggle
!kaggle datasets download -d odins0n/amex-parquet

^C
ERROR: Operation cancelled by user
  5%|█▉                                      | 416M/8.65G [00:02<00:32, 270MB/s]^C
  5%|█▉                                      | 418M/8.65G [00:02<00:41, 213MB/s]
User cancelled operation


In [61]:
!unzip "/kaggle/working/amex-parquet.zip"

Archive:  /kaggle/working/amex-parquet.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /kaggle/working/amex-parquet.zip or
        /kaggle/working/amex-parquet.zip.zip, and cannot find /kaggle/working/amex-parquet.zip.ZIP, period.


In [62]:
!rm "/kaggle/working/amex-parquet.zip"
gc.collect()

349

In [63]:
DATA_PATH = "/kaggle/working/train_data.parquet"
TEST_DATA_PATH = "/kaggle/working/test_data.parquet"
SAMPLE_SUBMISSION_PATH = "/kaggle/input/amex-default-prediction/sample_submission.csv"

### READ DATA

In [64]:
data_loaded = pd.read_parquet(DATA_PATH)

KeyboardInterrupt: 

In [ ]:
# Let's Keep Latest For Each Customer
data_sorted=data_loaded.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()
data_sorted = data_sorted.drop(['S_2'],axis=1)
del data_loaded
gc.collect()

In [ ]:
data_sorted.head()

In [ ]:
data_sorted.info()

In [ ]:
data_sorted.describe()

In [ ]:
gc.collect()

### FINDINGS

In [ ]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
target_cols = ['target']
numerical_cols = [num_col for num_col in data_sorted.columns if num_col not in cat_cols+target_cols ]


print("CAT COLS : ",str(cat_cols))
print("NUM COLS : ",str(numerical_cols))
print("TARGET COLS : ",str(target_cols))

### CHECK NAN AND DROP LARGELY MISSING COLUMNS

In [ ]:
total_rows = len(data_sorted)
threshold_to_remove_nan=total_rows * 0.75

CHECK NAN LARGE THAN THRES HOLD

In [ ]:
null_count = pd.DataFrame(data_sorted.isnull().sum(axis=0))
largely_missing_columns = list(null_count.loc[null_count[0]>threshold_to_remove_nan].index)
largely_missing_columns

In [ ]:
data_sorted = data_sorted.drop(largely_missing_columns,axis=1)

Update CAT,NUM List

In [ ]:
numerical_cols = [col for col in numerical_cols if col not in largely_missing_columns]
cat_cols = [col for col in cat_cols if col not in largely_missing_columns]

In [ ]:
gc.collect()

In [ ]:
data_sorted.describe()

### IMPUTING

In [ ]:
from sklearn.impute import SimpleImputer
imp_numerical = SimpleImputer(missing_values=np.nan, strategy='median')
imp_numerical.fit(data_sorted[numerical_cols])
data_sorted[numerical_cols] = imp_numerical.transform(data_sorted[numerical_cols])

In [ ]:
for col in cat_cols:
    data_sorted[col] =  data_sorted[col].fillna(data_sorted[col].mode()[0])

In [ ]:
data_sorted.head()

In [ ]:
null_count = pd.DataFrame(data_sorted.isnull().sum(axis=0))
list(null_count.loc[null_count[0]>0].index)

In [ ]:
gc.collect()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
data_sorted[cat_cols] = enc.fit_transform(data_sorted[cat_cols])

In [ ]:
data_sorted.head()

In [ ]:
corr = data_sorted.corr()
corelated_var= set()

for i in range(len(corr.columns)):
    for j in range(i):
        if(corr.iloc[i, j] > 0.9):
            colname = corr.columns[i]
            corelated_var.add(colname)

In [ ]:
corelated_var

In [ ]:
data_sorted = data_sorted.drop(corelated_var,axis=1)

In [ ]:
data_sorted.head()

In [ ]:
gc.collect()

# DATA PREPROCESSING TEST

In [ ]:
test_data_loaded = pd.read_parquet(TEST_DATA_PATH)

In [ ]:
# Let's Keep Latest For Each Customer
test_data_sorted=test_data_loaded.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()
test_data_sorted = test_data_sorted.drop(['S_2'],axis=1)
del test_data_loaded
gc.collect()

In [ ]:
test_data_sorted = test_data_sorted.drop(largely_missing_columns,axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
imp_numerical = SimpleImputer(missing_values=np.nan, strategy='median')
imp_numerical.fit(test_data_sorted[numerical_cols])
test_data_sorted[numerical_cols] = imp_numerical.transform(test_data_sorted[numerical_cols])

In [ ]:
for col in cat_cols:
    test_data_sorted[col] =  test_data_sorted[col].fillna(test_data_sorted[col].mode()[0])

In [ ]:
null_count = pd.DataFrame(test_data_sorted.isnull().sum(axis=0))
list(null_count.loc[null_count[0]>threshold_to_remove_nan].index)

In [ ]:
gc.collect()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
test_data_sorted[cat_cols] = enc.fit_transform(test_data_sorted[cat_cols])

In [ ]:
test_data_sorted = test_data_sorted.drop(corelated_var,axis=1)

In [ ]:
test_data_sorted.head()

In [ ]:
gc.collect()

In [ ]:
len(test_data_sorted)

# TRAIN MODEL

In [ ]:
all_feature_columns = list(data_sorted.columns)
all_feature_columns.remove('target')
X = data_sorted[all_feature_columns]
y = data_sorted['target']

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X, y, test_size=0.01, random_state=42, stratify=y)

In [ ]:
gc.collect()

In [88]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier, early_stopping
import xgboost as xgb

model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [89]:
y_pred = model.predict_proba(x_val)

In [90]:
y_pred_df = pd.DataFrame(y_pred[:,1],columns=["prediction"])
y_pred_df["customer_ID"]= x_val.index
y_pred_df.set_index("customer_ID",inplace=True)
y_pred_df

,prediction
customer_ID,
e7a323976e71c8def00040ba72577960db31b0847056ddec06a76725efe04574,0.000000
ac3d61c3dae86984d77b50c8552ec99eea7fc2cedbf16541c16a3934189ec804,0.000000
6d224f602013cf4013c7bd20fd8c0a60847aaf08e739eef24adc995d56d850f8,1.000000
60b2f51f9f7daebfc9132ec96ea4819c1aa95910a46b42a8374388d99dd608d3,0.666667
a7f5780dafc6d3095888db35ef907aedbd1a230e4ddd657e77333b4df211346f,0.000000
...,...
e468b316a006a1b14f0515741da78804583428e709a794f46ce57a4361b92a8a,0.666667
9013a31e4ea1a67440e40de27acb4e3105b26017dbeb3560938e20dcbcb9edea,1.000000
f5b826ebcfadfc4ab301daac5e1fd901104157f05bfe18ee357727f9cd3efccb,1.000000


In [91]:
y_val_df = pd.DataFrame(y_val)
y_val_df

,target
customer_ID,
e7a323976e71c8def00040ba72577960db31b0847056ddec06a76725efe04574,0
ac3d61c3dae86984d77b50c8552ec99eea7fc2cedbf16541c16a3934189ec804,0
6d224f602013cf4013c7bd20fd8c0a60847aaf08e739eef24adc995d56d850f8,1
60b2f51f9f7daebfc9132ec96ea4819c1aa95910a46b42a8374388d99dd608d3,1
a7f5780dafc6d3095888db35ef907aedbd1a230e4ddd657e77333b4df211346f,0
...,...
e468b316a006a1b14f0515741da78804583428e709a794f46ce57a4361b92a8a,0
9013a31e4ea1a67440e40de27acb4e3105b26017dbeb3560938e20dcbcb9edea,1
f5b826ebcfadfc4ab301daac5e1fd901104157f05bfe18ee357727f9cd3efccb,1


# EVAL METRICS

In [92]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        print(df)
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [93]:
amex_metric(y_val_df,y_pred_df)

                                                    target  prediction
customer_ID                                                           
ef22d61a81efdaf987e4ff81a0a5b6a72cf46663cee9994...       1         1.0
46c0f1342ea61f360dda9ee977d8d8fa84026a2afb7e0b8...       1         1.0
817b12c85220714b53972b87c780d0a5b41bf60f6c9c318...       1         1.0
a5429413dbaedf75d915749c11957046073a951d0a273c9...       1         1.0
eefcbc7c23b6a7fa134134ed07d9d33bf8b12905f4d2ee4...       1         1.0
...                                                    ...         ...
02f572e2117f8bdbae4eb5da9198a50dbb2e08cb032aa05...       0         0.0
7de9ff57e28cf8653860ab0f17047439b58ff9b78631ef9...       0         0.0
eb4e287099d835f08807ddcce858bd104a5a8a01bfc1ca1...       0         0.0
77915a916803d3d461c857a10965823823bb33bb72dc825...       0         0.0
1c96d8d9005b54f3f51f5215d1cf244fe7877c39253bf85...       0         0.0

[4590 rows x 2 columns]


0.6081103294438602

# PREDICT FOR TEST DATA

In [94]:
# CHUNK_SIZE = 10000
# y_test_pred = []
# for chunk_num in range(len(test_data_sorted) // CHUNK_SIZE + 1):
#     start_index = chunk_num*CHUNK_SIZE
#     end_index = min(chunk_num*CHUNK_SIZE + CHUNK_SIZE, len(test_data_sorted))
#     chunk = test_data_sorted[start_index:end_index]
    
#     preds = model.predict_proba(test_data_sorted)
#     y_test_pred = y_test_pred + list(preds[:,1])
#     print("\nChunk No : ",chunk_num)
#     print("Completed Predictions : ",len(y_test_pred))
#     print("Completed : ",str(end_index*100/len(test_data_sorted))," % ")


In [95]:
def chunker(seq, size):
        for pos in range(0, len(seq), size):
            yield seq.iloc[pos:pos + size] 


In [96]:
res=[]
chunk_size=1000
print(len(test_data_sorted)/chunk_size)
for idf in chunker(test_data_sorted, chunk_size):
    print(len(res))
    iy_test_pred = model.predict_proba(idf)
    res.append(iy_test_pred)

924.621
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
27

In [102]:
y_test_pred=[]
for i in range(len(res)):
    for val in res[i][:,1]:
        y_test_pred.append(val) 
    
len(y_test_pred)

924621

In [104]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': y_test_pred})
output

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.000000
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000000
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.000000
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.333333
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.000000
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.000000
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.333333
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.000000
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.333333


In [108]:
output.to_csv('submission-1.csv', index=False)

In [109]:
in1 = pd.read_csv('submission-1.csv')

In [110]:
in1.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.000000
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000000
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.000000
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.333333
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.000000
